In [1]:
import xarray as xr
xr.__version__

'0.10.0rc2'

In [9]:
from dask.distributed import Client
client = Client(scheduler_file='/home/02823/rabernat/scheduler.json')
client

Client Scheduler: tcp://129.114.58.132:8786 Dashboard: http://129.114.58.132:8787,Cluster Workers: 41 Cores: 164 Memory: 828.89 GB


In [3]:
ddir = '/gpfs/flash/users/rabernat/CM2.6/control/'

In [4]:
ds = xr.open_mfdataset(ddir + '*.nc', chunks={'st_ocean': 1, 'time':1})
ds

/home/02823/rabernat/miniconda3/envs/pangeo/lib/python3.6/site-packages/xarray/conventions.py:416: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy netCDF4.datetime objects instead, reason: dates out of range
  result = decode_cf_datetime(example_value, units, calendar)
/home/02823/rabernat/miniconda3/envs/pangeo/lib/python3.6/site-packages/xarray/conventions.py:435: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy netCDF4.datetime objects instead, reason: dates out of range
  calendar=self.calendar)


<xarray.Dataset>
Dimensions:         (nv: 2, st_edges_ocean: 51, st_ocean: 50, time: 146, xt_ocean: 3600, xu_ocean: 3600, yt_ocean: 2700, yu_ocean: 2700)
Coordinates:
  * xt_ocean        (xt_ocean) float64 -279.9 -279.8 -279.7 -279.6 -279.5 ...
  * yt_ocean        (yt_ocean) float64 -81.11 -81.07 -81.02 -80.98 -80.94 ...
  * st_ocean        (st_ocean) float64 5.034 15.1 25.22 35.36 45.58 55.85 ...
  * st_edges_ocean  (st_edges_ocean) float64 0.0 10.07 20.16 30.29 40.47 ...
  * nv              (nv) float64 1.0 2.0
  * time            (time) object  181-01-03 12:00:00  181-01-08 12:00:00 ...
  * xu_ocean        (xu_ocean) float64 -279.9 -279.8 -279.7 -279.6 -279.5 ...
  * yu_ocean        (yu_ocean) float64 -81.09 -81.05 -81.0 -80.96 -80.92 ...
Data variables:
    salt            (time, st_ocean, yt_ocean, xt_ocean) float64 dask.array<shape=(146, 50, 2700, 3600), chunksize=(1, 1, 2700, 3600)>
    average_T1      (time) object dask.array<shape=(146,), chunksize=(1,)>
    average_T2      (t

In [11]:
ds.nbytes / 1e9

2270.592108632

## Global Mean

In [12]:
# easy example...just take the mean of the whole dataset
# extremely simple and I/O bound
%time ds.mean().load()

CPU times: user 9.06 s, sys: 197 ms, total: 9.26 s
Wall time: 1min 49s


<xarray.Dataset>
Dimensions:      ()
Data variables:
    salt         float64 34.54
    average_DT   timedelta64[ns] 5 days
    time_bounds  timedelta64[ns] 303 days 23:28:23.308699
    temp         float64 5.778
    u            float64 0.005448
    v            float64 0.000825

## Monthly Climatology

Here we want to groupy by month and take the average in time.
This dataset uses a non-standard calendar, so `groupby('time.month')` doesn't work.
(xarray issues: [789](https://github.com/pydata/xarray/issues/789),
[1084](https://github.com/pydata/xarray/issues/1084),
[1252](https://github.com/pydata/xarray/pull/1252))


In [13]:
months = [val.month for val in ds.time.values]
ds.coords['month'] = (('time',), months)
ds

<xarray.Dataset>
Dimensions:         (nv: 2, st_edges_ocean: 51, st_ocean: 50, time: 146, xt_ocean: 3600, xu_ocean: 3600, yt_ocean: 2700, yu_ocean: 2700)
Coordinates:
  * xt_ocean        (xt_ocean) float64 -279.9 -279.8 -279.7 -279.6 -279.5 ...
  * yt_ocean        (yt_ocean) float64 -81.11 -81.07 -81.02 -80.98 -80.94 ...
  * st_ocean        (st_ocean) float64 5.034 15.1 25.22 35.36 45.58 55.85 ...
  * st_edges_ocean  (st_edges_ocean) float64 0.0 10.07 20.16 30.29 40.47 ...
  * nv              (nv) float64 1.0 2.0
  * time            (time) object  181-01-03 12:00:00  181-01-08 12:00:00 ...
  * xu_ocean        (xu_ocean) float64 -279.9 -279.8 -279.7 -279.6 -279.5 ...
  * yu_ocean        (yu_ocean) float64 -81.09 -81.05 -81.0 -80.96 -80.92 ...
    month           (time) int64 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3 4 4 4 4 ...
Data variables:
    salt            (time, st_ocean, yt_ocean, xt_ocean) float64 dask.array<shape=(146, 50, 2700, 3600), chunksize=(1, 1, 2700, 3600)>
    average_T1  

In [14]:
# persist monthly mean climatlogy to disk
# this ends up spilling lots of data to disk, which is okay on Wrangler
# because the scratch storage is so fast
ds_mm = ds.groupby('month').mean(dim='time').persist()

In [15]:
ds_mm

<xarray.Dataset>
Dimensions:         (month: 12, nv: 2, st_edges_ocean: 51, st_ocean: 50, xt_ocean: 3600, xu_ocean: 3600, yt_ocean: 2700, yu_ocean: 2700)
Coordinates:
  * xt_ocean        (xt_ocean) float64 -279.9 -279.8 -279.7 -279.6 -279.5 ...
  * yt_ocean        (yt_ocean) float64 -81.11 -81.07 -81.02 -80.98 -80.94 ...
  * st_ocean        (st_ocean) float64 5.034 15.1 25.22 35.36 45.58 55.85 ...
  * st_edges_ocean  (st_edges_ocean) float64 0.0 10.07 20.16 30.29 40.47 ...
  * nv              (nv) float64 1.0 2.0
  * xu_ocean        (xu_ocean) float64 -279.9 -279.8 -279.7 -279.6 -279.5 ...
  * yu_ocean        (yu_ocean) float64 -81.09 -81.05 -81.0 -80.96 -80.92 ...
  * month           (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    salt            (month, st_ocean, yt_ocean, xt_ocean) float64 dask.array<shape=(12, 50, 2700, 3600), chunksize=(1, 1, 2700, 3600)>
    average_DT      (month) timedelta64[ns] dask.array<shape=(12,), chunksize=(1,)>
    time_bounds     (month, nv

## Variance from monthly mean

This is the real science part of the calculation.
We look at the anomaly from the monthly climatology and calculate the variance of that anomaly.

In [16]:
ds_anom = ds.groupby('month') - ds_mm

In [18]:
ds_anom

<xarray.Dataset>
Dimensions:         (nv: 2, st_edges_ocean: 51, st_ocean: 50, time: 146, xt_ocean: 3600, xu_ocean: 3600, yt_ocean: 2700, yu_ocean: 2700)
Coordinates:
  * xt_ocean        (xt_ocean) float64 -279.9 -279.8 -279.7 -279.6 -279.5 ...
  * yt_ocean        (yt_ocean) float64 -81.11 -81.07 -81.02 -80.98 -80.94 ...
  * st_ocean        (st_ocean) float64 5.034 15.1 25.22 35.36 45.58 55.85 ...
  * st_edges_ocean  (st_edges_ocean) float64 0.0 10.07 20.16 30.29 40.47 ...
  * nv              (nv) float64 1.0 2.0
  * xu_ocean        (xu_ocean) float64 -279.9 -279.8 -279.7 -279.6 -279.5 ...
  * yu_ocean        (yu_ocean) float64 -81.09 -81.05 -81.0 -80.96 -80.92 ...
  * time            (time) object  181-01-03 12:00:00  181-01-08 12:00:00 ...
    month           (time) int64 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3 4 4 4 4 ...
Data variables:
    salt            (time, st_ocean, yt_ocean, xt_ocean) float64 dask.array<shape=(146, 50, 2700, 3600), chunksize=(146, 1, 2700, 3600)>
    average_DT

In [ ]:
# this don't ever finish computing
temp_var = (ds_anom.temp**2).mean(dim='time').load()

The above doesn't finish because workers keep dying. Sometimes it gets to the very end, with just a few tasks left, but it always dies.

```
distributed.worker - INFO - Can't find dependencies for key ('broadcast_to-mean_agg-aggregate-transpose-concatenate-cb1ded8a7edd185e51fdac6d1f13d799', 6, 6, 0, 0)
distributed.worker - ERROR - Worker stream died during communication: tcp://192.168.58.120:44101
Traceback (most recent call last):
  File "/home/02823/rabernat/miniconda3/envs/pangeo/lib/python3.6/site-packages/distributed/comm/tcp.py", line 170, in read
    n_frames = yield stream.read_bytes(8)
  File "/home/02823/rabernat/miniconda3/envs/pangeo/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/home/02823/rabernat/miniconda3/envs/pangeo/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
tornado.iostream.StreamClosedError: Stream is closed
```